In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
import os

import pandas as pd
# import pytorch_lightning as pl
import spacy
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchtext
from tqdm.notebook import tqdm

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

In [47]:
print(f"pytorch version: {torch.__version__}")
print(f"torchtext version: {torchtext.__version__}")

pytorch version: 1.4.0
torchtext version: 0.5.0


In [ ]:
# look at the data
train_filename = '/kaggle/input/nlp-getting-started/train.csv'
tr_df = pd.read_csv(train_filename, engine='python')
display(tr_df.head())
print(tr_df.shape)
print(tr_df.target.value_counts())

# Torchtext

In [ ]:
class spacy_tokenizer:
    def __init__(self, lang):
        self.lang = lang
        self.nlp = spacy.blank(self.lang)
        
    def __call__(self, text):
        return [token.text for token in self.nlp(text)]

tokenizer = spacy_tokenizer('en')
TEXT = torchtext.data.Field(tokenize=tokenizer, lower=True)
LABEL = torchtext.data.LabelField(use_vocab=False)

fields = [('id', None), ('keyword', None), ('location', None), ('text', TEXT), ('target', LABEL)]
dataset = torchtext.data.TabularDataset(path=train_filename, format='csv', fields=fields, skip_header=True)

In [ ]:
# split into train and validation dataset.
tr_data, val_data = dataset.split(split_ratio=0.8)
print(len(tr_data), len(val_data))

In [ ]:
# Build vocab
TEXT.build_vocab(tr_data)
print('num vocab', len(TEXT.vocab))

max_size = 20000
TEXT.build_vocab(tr_data, max_size=max_size)
print('num vocab', len(TEXT.vocab))

In [ ]:
# iterator
batch_size = 32
tr_iter = torchtext.data.BucketIterator(tr_data, batch_size=batch_size, shuffle=True, device=device)
val_iter = torchtext.data.BucketIterator(val_data, batch_size=batch_size, shuffle=False, device=device)  # do not use train=False

# The network

In [28]:
class Rnn(nn.Module):
    def __init__(self, input_size, embedding_dim, hidden_dim, output_size, bidirectional=False, n_layers=1, dropout=0, rnn_type='rnn'):
        super().__init__()
        self.bidirectional = bidirectional
        self.n_layers = n_layers
        self.rnn_type = rnn_type
        self.dropout = dropout
        
        self.embedding = nn.Embedding(input_size, embedding_dim)
        self.encoder = getattr(nn, rnn_type.upper())(embedding_dim, hidden_dim, bidirectional=bidirectional, num_layers=n_layers, dropout=dropout)
        
        in_features = hidden_dim
        if self.bidirectional:
            in_features *= 2
            
        self.fc = nn.Linear(in_features=in_features, out_features=output_size)
        
    def forward(self, x):
        x = self.embedding(x)
        if self.rnn_type.upper() == 'LSTM':
            x, (h_, c_) = self.encoder(x)
        else:
            x, h_ = self.encoder(x)

        x = self.fc(x[-1])
        
        return x

# Training

## RNN v.s. LSTM

In [29]:
# Single layer, unidirectional Vanilla RNN
input_size = len(TEXT.vocab)
embedding_dim = 100
hidden_dim = 100
output_size = 1
bidirectional = False
n_layers = 1
dropout = 0
rnn_type = 'rnn'
lr = 0.0001
num_epochs = 20

model = Rnn(input_size, embedding_dim, hidden_dim, output_size, bidirectional=bidirectional, n_layers=n_layers, dropout=dropout, rnn_type=rnn_type).to(device)
optimizer = optim.Adam(model.parameters(), lr=lr)
criterion = nn.BCEWithLogitsLoss()

for epoch in tqdm(range(num_epochs)):
    model.train()
    tr_loss = 0
    for idx, batch in enumerate(tr_iter):
        optimizer.zero_grad()
        
        x, y = batch
        
        logits = model(x)
        loss = criterion(logits.squeeze().to(float), y.to(float))
        tr_loss += loss.item()
        
        loss.backward()
        optimizer.step()
        
    tr_loss /= (idx + 1)
    
    with torch.no_grad():
        model.eval()
        num_correct = 0
        val_loss = 0
        for idx, batch in enumerate(val_iter):
            x, y = batch
            
            logits = model(x)
            loss = criterion(logits.squeeze().to(float), y.to(float))
            val_loss = loss.item()
            
            
            prediction = torch.sigmoid(logits)
            prediction = (prediction >= 0.5).to(int).squeeze()
            num_correct += sum(prediction == y)
            
        val_loss = loss / (idx + 1)
        accuracy = num_correct.item() / len(val_data) * 100
    
    print(f'End epoch {epoch + 1}: Train loss {tr_loss:.6f}, Validation loss {val_loss:.6f}, Accuracy {accuracy:.4f} %')

End epoch 1: Train loss 0.686143, Validation loss 0.015258, Accuracy 56.5332 %
End epoch 2: Train loss 0.684107, Validation loss 0.015248, Accuracy 56.5988 %
End epoch 3: Train loss 0.683151, Validation loss 0.015320, Accuracy 56.5332 %
End epoch 4: Train loss 0.682251, Validation loss 0.015335, Accuracy 56.3362 %
End epoch 5: Train loss 0.682430, Validation loss 0.015022, Accuracy 56.3362 %
End epoch 6: Train loss 0.681414, Validation loss 0.015875, Accuracy 56.3362 %
End epoch 7: Train loss 0.680970, Validation loss 0.015566, Accuracy 56.3362 %
End epoch 8: Train loss 0.681566, Validation loss 0.015377, Accuracy 56.4675 %
End epoch 9: Train loss 0.680608, Validation loss 0.015740, Accuracy 56.5988 %
End epoch 10: Train loss 0.680145, Validation loss 0.015657, Accuracy 56.4675 %
End epoch 11: Train loss 0.679780, Validation loss 0.015750, Accuracy 56.3362 %
End epoch 12: Train loss 0.679465, Validation loss 0.016013, Accuracy 56.2705 %
End epoch 13: Train loss 0.679475, Validation los

In [31]:
# Single layer, unidirectional RNN with LSTM cell.
input_size = len(TEXT.vocab)
embedding_dim = 100
hidden_dim = 100
output_size = 1
bidirectional = False
n_layers = 1
dropout = 0
rnn_type = 'lstm'
lr = 0.0001
num_epochs = 20

model = Rnn(input_size, embedding_dim, hidden_dim, output_size, bidirectional=bidirectional, n_layers=n_layers, dropout=dropout, rnn_type=rnn_type).to(device)
optimizer = optim.Adam(model.parameters(), lr=lr)
criterion = nn.BCEWithLogitsLoss()

for epoch in tqdm(range(num_epochs)):
    model.train()
    tr_loss = 0
    for idx, batch in enumerate(tr_iter):
        optimizer.zero_grad()
        
        x, y = batch
        
        logits = model(x)
        loss = criterion(logits.squeeze().to(float), y.to(float))
        tr_loss += loss.item()
        
        loss.backward()
        optimizer.step()
        
    tr_loss /= (idx + 1)
    
    with torch.no_grad():
        model.eval()
        num_correct = 0
        val_loss = 0
        for idx, batch in enumerate(val_iter):
            x, y = batch
            
            logits = model(x)
            loss = criterion(logits.squeeze().to(float), y.to(float))
            val_loss = loss.item()
            
            
            prediction = torch.sigmoid(logits)
            prediction = (prediction >= 0.5).to(int).squeeze()
            num_correct += sum(prediction == y)
            
        val_loss = loss / (idx + 1)
        accuracy = num_correct.item() / len(val_data) * 100
    
    print(f'End epoch {epoch + 1}: Train loss {tr_loss:.6f}, Validation loss {val_loss:.6f}, Accuracy {accuracy:.4f} %')

End epoch 1: Train loss 0.686149, Validation loss 0.015434, Accuracy 56.1392 %
End epoch 2: Train loss 0.683519, Validation loss 0.015391, Accuracy 56.2049 %
End epoch 3: Train loss 0.681743, Validation loss 0.015376, Accuracy 56.4018 %
End epoch 4: Train loss 0.670921, Validation loss 0.015600, Accuracy 60.2101 %
End epoch 5: Train loss 0.659021, Validation loss 0.014993, Accuracy 64.4123 %
End epoch 6: Train loss 0.625467, Validation loss 0.013794, Accuracy 64.5437 %
End epoch 7: Train loss 0.585155, Validation loss 0.013000, Accuracy 66.5135 %
End epoch 8: Train loss 0.549042, Validation loss 0.012691, Accuracy 67.1701 %
End epoch 9: Train loss 0.540150, Validation loss 0.011936, Accuracy 69.4682 %
End epoch 10: Train loss 0.513276, Validation loss 0.011784, Accuracy 67.4327 %
End epoch 11: Train loss 0.493698, Validation loss 0.012312, Accuracy 68.3519 %
End epoch 12: Train loss 0.470956, Validation loss 0.012282, Accuracy 68.8116 %
End epoch 13: Train loss 0.456053, Validation los

## Number of layers

In [33]:
# Triple layer, biidirectional RNN with LSTM cell.
input_size = len(TEXT.vocab)
embedding_dim = 100
hidden_dim = 100
output_size = 1
bidirectional = True
n_layers = 3
dropout = 0
rnn_type = 'lstm'
lr = 0.0001
num_epochs = 20

model = Rnn(input_size, embedding_dim, hidden_dim, output_size, bidirectional=bidirectional, n_layers=n_layers, dropout=dropout, rnn_type=rnn_type).tcvvo(device)
optimizer = optim.Adam(model.parameters(), lr=lr)
criterion = nn.BCEWithLogitsLoss()

for epoch in tqdm(range(num_epochs)):
    model.train()
    tr_loss = 0
    for idx, batch in enumerate(tr_iter):
        optimizer.zero_grad()
        
        x, y = batch
        
        logits = model(x)
        loss = criterion(logits.squeeze().to(float), y.to(float))
        tr_loss += loss.item()
        
        loss.backward()
        optimizer.step()
        
    tr_loss /= (idx + 1)
    
    with torch.no_grad():
        model.eval()
        num_correct = 0
        val_loss = 0
        for idx, batch in enumerate(val_iter):
            x, y = batch
            
            logits = model(x)
            loss = criterion(logits.squeeze().to(float), y.to(float))
            val_loss = loss.item()
            
            
            prediction = torch.sigmoid(logits)
            prediction = (prediction >= 0.5).to(int).squeeze()
            num_correct += sum(prediction == y)
            
        val_loss = loss / (idx + 1)
        accuracy = num_correct.item() / len(val_data) * 100
    
    print(f'End epoch {epoch + 1}: Train loss {tr_loss:.6f}, Validation loss {val_loss:.6f}, Accuracy {accuracy:.4f} %')

End epoch 1: Train loss 0.686180, Validation loss 0.015408, Accuracy 56.5988 %
End epoch 2: Train loss 0.682783, Validation loss 0.014874, Accuracy 56.5988 %
End epoch 3: Train loss 0.654599, Validation loss 0.013325, Accuracy 67.4984 %
End epoch 4: Train loss 0.604910, Validation loss 0.012273, Accuracy 71.1097 %
End epoch 5: Train loss 0.573841, Validation loss 0.012418, Accuracy 72.2915 %
End epoch 6: Train loss 0.542105, Validation loss 0.013624, Accuracy 71.7663 %
End epoch 7: Train loss 0.518034, Validation loss 0.012863, Accuracy 71.8976 %
End epoch 8: Train loss 0.489685, Validation loss 0.013815, Accuracy 73.8674 %
End epoch 9: Train loss 0.471655, Validation loss 0.013798, Accuracy 73.4077 %
End epoch 10: Train loss 0.445759, Validation loss 0.014342, Accuracy 71.5693 %
End epoch 11: Train loss 0.419717, Validation loss 0.014549, Accuracy 73.2764 %
End epoch 12: Train loss 0.398545, Validation loss 0.015164, Accuracy 73.3421 %
End epoch 13: Train loss 0.375532, Validation los

In [38]:
# 5 layers, biidirectional RNN with LSTM cell.
input_size = len(TEXT.vocab)
embedding_dim = 100
hidden_dim = 100
output_size = 1
bidirectional = True
n_layers = 3
dropout = 0
rnn_type = 'lstm'
lr = 0.0001
num_epochs = 20

model = Rnn(input_size, embedding_dim, hidden_dim, output_size, bidirectional=bidirectional, n_layers=n_layers, dropout=dropout, rnn_type=rnn_type).to(device)
optimizer = optim.Adam(model.parameters(), lr=lr)
criterion = nn.BCEWithLogitsLoss()

for epoch in tqdm(range(num_epochs)):
    model.train()
    tr_loss = 0
    for idx, batch in enumerate(tr_iter):
        optimizer.zero_grad()
        
        x, y = batch
        
        logits = model(x)
        loss = criterion(logits.squeeze().to(float), y.to(float))
        tr_loss += loss.item()
        
        loss.backward()
        optimizer.step()
        
    tr_loss /= (idx + 1)
    
    with torch.no_grad():
        model.eval()
        num_correct = 0
        val_loss = 0
        for idx, batch in enumerate(val_iter):
            x, y = batch
            
            logits = model(x)
            loss = criterion(logits.squeeze().to(float), y.to(float))
            val_loss = loss.item()
            
            
            prediction = torch.sigmoid(logits)
            prediction = (prediction >= 0.5).to(int).squeeze()
            num_correct += sum(prediction == y)
            
        val_loss = loss / (idx + 1)
        accuracy = num_correct.item() / len(val_data) * 100
    
    print(f'End epoch {epoch + 1}: Train loss {tr_loss:.6f}, Validation loss {val_loss:.6f}, Accuracy {accuracy:.4f} %')

End epoch 1: Train loss 0.684374, Validation loss 0.015940, Accuracy 56.5988 %
End epoch 2: Train loss 0.682356, Validation loss 0.015918, Accuracy 56.5988 %
End epoch 3: Train loss 0.665244, Validation loss 0.018898, Accuracy 57.3867 %
End epoch 4: Train loss 0.620694, Validation loss 0.017384, Accuracy 66.1195 %
End epoch 5: Train loss 0.576020, Validation loss 0.017279, Accuracy 67.3014 %
End epoch 6: Train loss 0.556717, Validation loss 0.018272, Accuracy 67.5640 %
End epoch 7: Train loss 0.533663, Validation loss 0.016539, Accuracy 69.6651 %
End epoch 8: Train loss 0.514650, Validation loss 0.018890, Accuracy 68.4176 %
End epoch 9: Train loss 0.494808, Validation loss 0.018009, Accuracy 69.2712 %
End epoch 10: Train loss 0.480799, Validation loss 0.015229, Accuracy 72.7511 %
End epoch 11: Train loss 0.465374, Validation loss 0.018371, Accuracy 70.9783 %
End epoch 12: Train loss 0.451322, Validation loss 0.019400, Accuracy 69.2055 %
End epoch 13: Train loss 0.431649, Validation los

In [39]:
# 7 layers, biidirectional RNN with LSTM cell.
input_size = len(TEXT.vocab)
embedding_dim = 100
hidden_dim = 100
output_size = 1
bidirectional = True
n_layers = 3
dropout = 0
rnn_type = 'lstm'
lr = 0.0001
num_epochs = 20

model = Rnn(input_size, embedding_dim, hidden_dim, output_size, bidirectional=bidirectional, n_layers=n_layers, dropout=dropout, rnn_type=rnn_type).to(device)
optimizer = optim.Adam(model.parameters(), lr=lr)
criterion = nn.BCEWithLogitsLoss()

for epoch in tqdm(range(num_epochs)):
    model.train()
    tr_loss = 0
    for idx, batch in enumerate(tr_iter):
        optimizer.zero_grad()
        
        x, y = batch
        
        logits = model(x)
        loss = criterion(logits.squeeze().to(float), y.to(float))
        tr_loss += loss.item()
        
        loss.backward()
        optimizer.step()
        
    tr_loss /= (idx + 1)
    
    with torch.no_grad():
        model.eval()
        num_correct = 0
        val_loss = 0
        for idx, batch in enumerate(val_iter):
            x, y = batch
            
            logits = model(x)
            loss = criterion(logits.squeeze().to(float), y.to(float))
            val_loss = loss.item()
            
            
            prediction = torch.sigmoid(logits)
            prediction = (prediction >= 0.5).to(int).squeeze()
            num_correct += sum(prediction == y)
            
        val_loss = loss / (idx + 1)
        accuracy = num_correct.item() / len(val_data) * 100
    
    print(f'End epoch {epoch + 1}: Train loss {tr_loss:.6f}, Validation loss {val_loss:.6f}, Accuracy {accuracy:.4f} %')

End epoch 1: Train loss 0.684439, Validation loss 0.015396, Accuracy 56.5988 %
End epoch 2: Train loss 0.681865, Validation loss 0.014990, Accuracy 56.5988 %
End epoch 3: Train loss 0.667820, Validation loss 0.014766, Accuracy 67.3670 %
End epoch 4: Train loss 0.615748, Validation loss 0.011910, Accuracy 69.2712 %
End epoch 5: Train loss 0.590287, Validation loss 0.011234, Accuracy 71.5036 %
End epoch 6: Train loss 0.568022, Validation loss 0.012401, Accuracy 73.5391 %
End epoch 7: Train loss 0.576193, Validation loss 0.015705, Accuracy 71.1097 %
End epoch 8: Train loss 0.578437, Validation loss 0.014177, Accuracy 71.4380 %
End epoch 9: Train loss 0.565581, Validation loss 0.012663, Accuracy 70.7157 %
End epoch 10: Train loss 0.580068, Validation loss 0.013803, Accuracy 67.8267 %
End epoch 11: Train loss 0.571825, Validation loss 0.014175, Accuracy 71.1753 %
End epoch 12: Train loss 0.569560, Validation loss 0.012805, Accuracy 64.4780 %
End epoch 13: Train loss 0.572228, Validation los

## Embedding Dimensions

In [43]:
# Triple layer, biidirectional RNN with LSTM cell.
input_size = len(TEXT.vocab)
embedding_dim = 300
hidden_dim = 100
output_size = 1
bidirectional = True
n_layers = 3
dropout = 0
rnn_type = 'lstm'
lr = 0.0001
num_epochs = 20

model = Rnn(input_size, embedding_dim, hidden_dim, output_size, bidirectional=bidirectional, n_layers=n_layers, dropout=dropout, rnn_type=rnn_type).to(device)
optimizer = optim.Adam(model.parameters(), lr=lr)
criterion = nn.BCEWithLogitsLoss()

for epoch in tqdm(range(num_epochs)):
    model.train()
    tr_loss = 0
    for idx, batch in enumerate(tr_iter):
        optimizer.zero_grad()
        
        x, y = batch
        
        logits = model(x)
        loss = criterion(logits.squeeze().to(float), y.to(float))
        tr_loss += loss.item()
        
        loss.backward()
        optimizer.step()
        
    tr_loss /= (idx + 1)
    
    with torch.no_grad():
        model.eval()
        num_correct = 0
        val_loss = 0
        for idx, batch in enumerate(val_iter):
            x, y = batch
            
            logits = model(x)
            loss = criterion(logits.squeeze().to(float), y.to(float))
            val_loss = loss.item()
            
            
            prediction = torch.sigmoid(logits)
            prediction = (prediction >= 0.5).to(int).squeeze()
            num_correct += sum(prediction == y)
            
        val_loss = loss / (idx + 1)
        accuracy = num_correct.item() / len(val_data) * 100
    
    print(f'End epoch {epoch + 1}: Train loss {tr_loss:.6f}, Validation loss {val_loss:.6f}, Accuracy {accuracy:.4f} %')

End epoch 1: Train loss 0.685582, Validation loss 0.015646, Accuracy 56.5988 %
End epoch 2: Train loss 0.680227, Validation loss 0.015636, Accuracy 56.7301 %
End epoch 3: Train loss 0.650998, Validation loss 0.016594, Accuracy 61.1293 %
End epoch 4: Train loss 0.565917, Validation loss 0.014101, Accuracy 68.4176 %
End epoch 5: Train loss 0.481096, Validation loss 0.013798, Accuracy 71.6349 %
End epoch 6: Train loss 0.419266, Validation loss 0.012927, Accuracy 74.9836 %
End epoch 7: Train loss 0.362948, Validation loss 0.013514, Accuracy 73.6704 %
End epoch 8: Train loss 0.311605, Validation loss 0.014887, Accuracy 75.7058 %
End epoch 9: Train loss 0.272454, Validation loss 0.015892, Accuracy 75.6402 %
End epoch 10: Train loss 0.249996, Validation loss 0.015541, Accuracy 75.1149 %
End epoch 11: Train loss 0.226640, Validation loss 0.015989, Accuracy 74.8523 %
End epoch 12: Train loss 0.220490, Validation loss 0.015892, Accuracy 74.3926 %
End epoch 13: Train loss 0.202969, Validation los

In [42]:
# Triple layer, biidirectional RNN with LSTM cell.
input_size = len(TEXT.vocab)
embedding_dim = 500
hidden_dim = 100
output_size = 1
bidirectional = True
n_layers = 3
dropout = 0
rnn_type = 'lstm'
lr = 0.0001
num_epochs = 20

model = Rnn(input_size, embedding_dim, hidden_dim, output_size, bidirectional=bidirectional, n_layers=n_layers, dropout=dropout, rnn_type=rnn_type).to(device)
optimizer = optim.Adam(model.parameters(), lr=lr)
criterion = nn.BCEWithLogitsLoss()

for epoch in tqdm(range(num_epochs)):
    model.train()
    tr_loss = 0
    for idx, batch in enumerate(tr_iter):
        optimizer.zero_grad()
        
        x, y = batch
        
        logits = model(x)
        loss = criterion(logits.squeeze().to(float), y.to(float))
        tr_loss += loss.item()
        
        loss.backward()
        optimizer.step()
        
    tr_loss /= (idx + 1)
    
    with torch.no_grad():
        model.eval()
        num_correct = 0
        val_loss = 0
        for idx, batch in enumerate(val_iter):
            x, y = batch
            
            logits = model(x)
            loss = criterion(logits.squeeze().to(float), y.to(float))
            val_loss = loss.item()
            
            
            prediction = torch.sigmoid(logits)
            prediction = (prediction >= 0.5).to(int).squeeze()
            num_correct += sum(prediction == y)
            
        val_loss = loss / (idx + 1)
        accuracy = num_correct.item() / len(val_data) * 100
    
    print(f'End epoch {epoch + 1}: Train loss {tr_loss:.6f}, Validation loss {val_loss:.6f}, Accuracy {accuracy:.4f} %')

End epoch 1: Train loss 0.684637, Validation loss 0.015275, Accuracy 56.5988 %
End epoch 2: Train loss 0.673158, Validation loss 0.015994, Accuracy 61.7859 %
End epoch 3: Train loss 0.612146, Validation loss 0.016260, Accuracy 70.9127 %
End epoch 4: Train loss 0.520551, Validation loss 0.013961, Accuracy 74.3270 %
End epoch 5: Train loss 0.439461, Validation loss 0.013831, Accuracy 75.6402 %
End epoch 6: Train loss 0.359533, Validation loss 0.013396, Accuracy 75.5745 %
End epoch 7: Train loss 0.309813, Validation loss 0.015710, Accuracy 75.6402 %
End epoch 8: Train loss 0.271392, Validation loss 0.016900, Accuracy 76.3624 %
End epoch 9: Train loss 0.236591, Validation loss 0.014860, Accuracy 76.0998 %
End epoch 10: Train loss 0.224945, Validation loss 0.021270, Accuracy 74.9836 %
End epoch 11: Train loss 0.197351, Validation loss 0.018441, Accuracy 76.2968 %
End epoch 12: Train loss 0.186057, Validation loss 0.018582, Accuracy 76.8221 %
End epoch 13: Train loss 0.177162, Validation los

## Hidden Dim

In [44]:
# Triple layer, biidirectional RNN with LSTM cell.
input_size = len(TEXT.vocab)
embedding_dim = 500
hidden_dim = 300
output_size = 1
bidirectional = True
n_layers = 3
dropout = 0
rnn_type = 'lstm'
lr = 0.0001
num_epochs = 20

model = Rnn(input_size, embedding_dim, hidden_dim, output_size, bidirectional=bidirectional, n_layers=n_layers, dropout=dropout, rnn_type=rnn_type).to(device)
optimizer = optim.Adam(model.parameters(), lr=lr)
criterion = nn.BCEWithLogitsLoss()

for epoch in tqdm(range(num_epochs)):
    model.train()
    tr_loss = 0
    for idx, batch in enumerate(tr_iter):
        optimizer.zero_grad()
        
        x, y = batch
        
        logits = model(x)
        loss = criterion(logits.squeeze().to(float), y.to(float))
        tr_loss += loss.item()
        
        loss.backward()
        optimizer.step()
        
    tr_loss /= (idx + 1)
    
    with torch.no_grad():
        model.eval()
        num_correct = 0
        val_loss = 0
        for idx, batch in enumerate(val_iter):
            x, y = batch
            
            logits = model(x)
            loss = criterion(logits.squeeze().to(float), y.to(float))
            val_loss = loss.item()
            
            
            prediction = torch.sigmoid(logits)
            prediction = (prediction >= 0.5).to(int).squeeze()
            num_correct += sum(prediction == y)
            
        val_loss = loss / (idx + 1)
        accuracy = num_correct.item() / len(val_data) * 100
    
    print(f'End epoch {epoch + 1}: Train loss {tr_loss:.6f}, Validation loss {val_loss:.6f}, Accuracy {accuracy:.4f} %')

End epoch 1: Train loss 0.683744, Validation loss 0.015565, Accuracy 56.5988 %
End epoch 2: Train loss 0.632728, Validation loss 0.014105, Accuracy 71.5693 %
End epoch 3: Train loss 0.511221, Validation loss 0.011944, Accuracy 77.0190 %
End epoch 4: Train loss 0.398272, Validation loss 0.010631, Accuracy 76.2968 %
End epoch 5: Train loss 0.321018, Validation loss 0.014552, Accuracy 77.2160 %
End epoch 6: Train loss 0.247653, Validation loss 0.012144, Accuracy 77.8726 %
End epoch 7: Train loss 0.220850, Validation loss 0.015627, Accuracy 76.8877 %
End epoch 8: Train loss 0.182591, Validation loss 0.015839, Accuracy 75.7715 %
End epoch 9: Train loss 0.156471, Validation loss 0.017419, Accuracy 76.4938 %
End epoch 10: Train loss 0.135555, Validation loss 0.013744, Accuracy 76.6907 %
End epoch 11: Train loss 0.117362, Validation loss 0.011401, Accuracy 76.0998 %
End epoch 12: Train loss 0.102122, Validation loss 0.017444, Accuracy 77.2160 %
End epoch 13: Train loss 0.088683, Validation los

In [45]:
# Triple layer, biidirectional RNN with LSTM cell.
input_size = len(TEXT.vocab)
embedding_dim = 500
hidden_dim = 500
output_size = 1
bidirectional = True
n_layers = 3
dropout = 0
rnn_type = 'lstm'
lr = 0.0001
num_epochs = 20

model = Rnn(input_size, embedding_dim, hidden_dim, output_size, bidirectional=bidirectional, n_layers=n_layers, dropout=dropout, rnn_type=rnn_type).to(device)
optimizer = optim.Adam(model.parameters(), lr=lr)
criterion = nn.BCEWithLogitsLoss()

for epoch in tqdm(range(num_epochs)):
    model.train()
    tr_loss = 0
    for idx, batch in enumerate(tr_iter):
        optimizer.zero_grad()
        
        x, y = batch
        
        logits = model(x)
        loss = criterion(logits.squeeze().to(float), y.to(float))
        tr_loss += loss.item()
        
        loss.backward()
        optimizer.step()
        
    tr_loss /= (idx + 1)
    
    with torch.no_grad():
        model.eval()
        num_correct = 0
        val_loss = 0
        for idx, batch in enumerate(val_iter):
            x, y = batch
            
            logits = model(x)
            loss = criterion(logits.squeeze().to(float), y.to(float))
            val_loss = loss.item()
            
            
            prediction = torch.sigmoid(logits)
            prediction = (prediction >= 0.5).to(int).squeeze()
            num_correct += sum(prediction == y)
            
        val_loss = loss / (idx + 1)
        accuracy = num_correct.item() / len(val_data) * 100
    
    print(f'End epoch {epoch + 1}: Train loss {tr_loss:.6f}, Validation loss {val_loss:.6f}, Accuracy {accuracy:.4f} %')

End epoch 1: Train loss 0.684893, Validation loss 0.014801, Accuracy 56.5988 %
End epoch 2: Train loss 0.666376, Validation loss 0.012886, Accuracy 69.4682 %
End epoch 3: Train loss 0.537706, Validation loss 0.012002, Accuracy 75.0492 %
End epoch 4: Train loss 0.417694, Validation loss 0.010023, Accuracy 76.0341 %
End epoch 5: Train loss 0.328530, Validation loss 0.011392, Accuracy 75.9028 %
End epoch 6: Train loss 0.262488, Validation loss 0.009441, Accuracy 76.9534 %
End epoch 7: Train loss 0.199080, Validation loss 0.011625, Accuracy 76.7564 %
End epoch 8: Train loss 0.158422, Validation loss 0.014837, Accuracy 74.3270 %
End epoch 9: Train loss 0.113986, Validation loss 0.017211, Accuracy 75.4432 %
End epoch 10: Train loss 0.096302, Validation loss 0.019391, Accuracy 76.2311 %
End epoch 11: Train loss 0.067376, Validation loss 0.020576, Accuracy 74.9836 %
End epoch 12: Train loss 0.065516, Validation loss 0.032031, Accuracy 72.2915 %
End epoch 13: Train loss 0.055510, Validation los

In [46]:
# Triple layer, biidirectional RNN with LSTM cell.
input_size = len(TEXT.vocab)
embedding_dim = 500
hidden_dim = 700
output_size = 1
bidirectional = True
n_layers = 3
dropout = 0
rnn_type = 'lstm'
lr = 0.0001
num_epochs = 20

model = Rnn(input_size, embedding_dim, hidden_dim, output_size, bidirectional=bidirectional, n_layers=n_layers, dropout=dropout, rnn_type=rnn_type).to(device)
optimizer = optim.Adam(model.parameters(), lr=lr)
criterion = nn.BCEWithLogitsLoss()

for epoch in tqdm(range(num_epochs)):
    model.train()
    tr_loss = 0
    for idx, batch in enumerate(tr_iter):
        optimizer.zero_grad()
        
        x, y = batch
        
        logits = model(x)
        loss = criterion(logits.squeeze().to(float), y.to(float))
        tr_loss += loss.item()
        
        loss.backward()
        optimizer.step()
        
    tr_loss /= (idx + 1)
    
    with torch.no_grad():
        model.eval()
        num_correct = 0
        val_loss = 0
        for idx, batch in enumerate(val_iter):
            x, y = batch
            
            logits = model(x)
            loss = criterion(logits.squeeze().to(float), y.to(float))
            val_loss = loss.item()
            
            
            prediction = torch.sigmoid(logits)
            prediction = (prediction >= 0.5).to(int).squeeze()
            num_correct += sum(prediction == y)
            
        val_loss = loss / (idx + 1)
        accuracy = num_correct.item() / len(val_data) * 100
    
    print(f'End epoch {epoch + 1}: Train loss {tr_loss:.6f}, Validation loss {val_loss:.6f}, Accuracy {accuracy:.4f} %')

End epoch 1: Train loss 0.684151, Validation loss 0.014847, Accuracy 56.6645 %
End epoch 2: Train loss 0.627720, Validation loss 0.017618, Accuracy 67.6953 %
End epoch 3: Train loss 0.538095, Validation loss 0.015783, Accuracy 76.4281 %
End epoch 4: Train loss 0.438291, Validation loss 0.016437, Accuracy 75.1149 %
End epoch 5: Train loss 0.356982, Validation loss 0.012747, Accuracy 76.5594 %
End epoch 6: Train loss 0.296356, Validation loss 0.015456, Accuracy 77.0847 %
End epoch 7: Train loss 0.219451, Validation loss 0.022575, Accuracy 77.3473 %
End epoch 8: Train loss 0.177110, Validation loss 0.015675, Accuracy 77.6756 %
End epoch 9: Train loss 0.155142, Validation loss 0.016791, Accuracy 76.2968 %
End epoch 10: Train loss 0.122729, Validation loss 0.015252, Accuracy 77.6756 %
End epoch 11: Train loss 0.103109, Validation loss 0.018042, Accuracy 77.0190 %
End epoch 12: Train loss 0.087596, Validation loss 0.026503, Accuracy 76.0998 %
End epoch 13: Train loss 0.081479, Validation los

# Summary of Performance

|RNN Cell|num_layers|directionality|Embedding_dim|Hidden_dim|validation accuracy|
|---|---|---|---|---|
|RNN|1|unidirectional|100|100|66.25 %|
|LSTM|1|unidirectional|100|100|69.34%|
|LSTM|3|bidirectional|100|100|75.57%|
|LSTM|5|bidirectional|100|100|72.75%|
|LSTM|7|bidirectional|100|100|73.54%|
|LSTM|3|bidirectional|300|100|75.71%|
|LSTM|3|bidirectional|500|100|77.02%|
|**LSTM\***|**3**|**bidirectional**|**500**|**300**|**78.39%** (14 epochs)|
|LSTM|3|bidirectional|500|500|76.23%|
|LSTM|3|bidirectional|500|700|77.68%|